In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
import re

In [ ]:
model = ChatOllama(model="llama3.2", base_url="http://localhost:11434", temperature=0)
prompt = PromptTemplate(
    input_variables=["question"],
    template="You are a helpful assistant that helps people find information. "
    "You will be given a question, and you will provide a concise answer. \n{question}"
    )
messages = prompt.format(question="What is the capital of France?")
print(messages)

In [ ]:
response = model.invoke(messages).content
# response = re.sub(r"<think>.*?</think>", "", response, flags=re.DOTALL).strip()
print(response)

In [ ]:
history = [SystemMessage(content="You are a helpful Order Tracking System for himalaya  and responds with very concised answers.")]
while True:
    user_input = input("User: ")
    print(f"User: {user_input}")
    if user_input.lower() in ["exit", "quit"]:
        break
    history.append(HumanMessage(content=user_input))
    response = model.invoke(history).content
    # response = re.sub(r"<think>.*?</think>", "", response, flags=re.DOTALL).strip()
    print(f"AI: {response}")
    history.append(AIMessage(content=response))


In [18]:
# pip install langchain langchain-community pydantic

import re
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain_community.chat_models import ChatOllama
from langchain.tools import tool

# -----------------------
# Demo "Database"
# -----------------------
ORDERS = {
    "1001": {"status": "processing", "location": "Main Warehouse", "can_cancel": True},
    "1002": {"status": "shipped", "location": "Regional Hub", "can_cancel": True},
    "1003": {"status": "out for delivery", "location": "City Hub", "can_cancel": False},
    "1004": {"status": "delivered", "location": "Customer Address", "can_cancel": False},
}

# -----------------------
# Utility
# -----------------------
def clean_order_id(raw: str) -> str:
    """Extract numeric order ID from input string."""
    match = re.search(r"\d+", raw)
    return match.group(0) if match else raw.strip()

# -----------------------
# Schemas
# -----------------------
class OrderInput(BaseModel):
    order_id: str = Field(..., description="The numeric order ID as a string")

# -----------------------
# Tools
# -----------------------
@tool("get_order_status", args_schema=OrderInput, return_direct=True)
def get_order_status(order_id: str) -> str:
    """Get the current status and location of an order."""
    order_id = clean_order_id(order_id)
    order = ORDERS.get(order_id)
    if not order:
        return f"Order {order_id} not found."
    return f"Order {order_id} is {order['status']} at {order['location']}."

@tool("cancel_order", args_schema=OrderInput, return_direct=True)
def cancel_order(order_id: str) -> str:
    """Cancel an order if possible."""
    order_id = clean_order_id(order_id)
    order = ORDERS.get(order_id)
    if not order:
        return f"Order {order_id} not found."
    if not order["can_cancel"]:
        return f"Order {order_id} cannot be canceled (already {order['status']})."
    order["status"] = "canceled"
    order["can_cancel"] = False
    return f"Order {order_id} has been canceled."

@tool("track_order", args_schema=OrderInput, return_direct=True)
def track_order(order_id: str) -> str:
    """Track the live location of an order."""
    order_id = clean_order_id(order_id)
    order = ORDERS.get(order_id)
    if not order:
        return f"Order {order_id} not found."
    return f"Order {order_id} is currently at {order['location']}."

# -----------------------
# LLM + Agent
# -----------------------
llm = ChatOllama(
    model="llama3.2",
    temperature=0,
    system=(
        "You are an order tracking assistant. "
        "If the user does not provide an order ID, you must ask them to provide one before calling any tool."
    )
)

tools = [get_order_status, cancel_order, track_order]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    max_iterations=5,  # limit reasoning steps
    verbose=True
)

# -----------------------
# Demo
# -----------------------
print("\n=== DEMO: Order Tracking Agent (Max 2 Steps) ===\n")

queries = [
    "Check the status of order 1001",
    "Where is order 1002 right now?",
    "Cancel order 1003",
    "Cancel order 1002",
    "Get the status of order 1002 after cancellation",
    "Track order 1004",
    "Where is my order?",  # should ask for ID now
]

for q in queries:
    print(f"\nUser: {q}")
    response = agent.run(q)
    print(f"Agent: {response}")



=== DEMO: Order Tracking Agent (Max 2 Steps) ===


User: Check the status of order 1001


> Entering new AgentExecutor chain...
Action:
```
{
  "action": "get_order_status",
  "action_input": {"order_id": "1001"}
}
```

Observation: Order 1001 is processing at Main Warehouse.


> Finished chain.
Agent: Order 1001 is processing at Main Warehouse.

User: Where is order 1002 right now?


> Entering new AgentExecutor chain...
Action:
```
{
  "action": "get_order_status",
  "action_input": {"order_id": "1002"}
}
```

Observation: Order 1002 is shipped at Regional Hub.


> Finished chain.
Agent: Order 1002 is shipped at Regional Hub.

User: Cancel order 1003


> Entering new AgentExecutor chain...
Action:
```
{
  "action": "cancel_order",
  "action_input": {"order_id": "1003"}
}
```

Observation: Order 1003 cannot be canceled (already out for delivery).


> Finished chain.
Agent: Order 1003 cannot be canceled (already out for delivery).

User: Cancel order 1002


> Entering new AgentExecuto

In [ ]:
ORDERS

In [ ]:
while True:
    user_input = input("User: ")
    print(f"User: {user_input}")
    if user_input.lower() in ["exit", "quit"]:
        break

    response = agent.run(user_input)
    print(f"Agent: {response}") 